In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lcs import *
import networkx as nx
from scipy.stats import beta
from numpy.linalg import eigh

In [ ]:
A = zkc()
# A = erdos_renyi(50, 0.5)
n = A.shape[0]

rho0 = 1
tmax = 1000
gamma = 0.1
x0 = np.zeros(n)
x0[random.sample(range(n), k=int(n*rho0))] = 1

In [ ]:
x0 = np.zeros(n)
x0[random.randrange(n)] = 1

nu = eigh(A)[0][-1]
b = 3 * gamma / nu

# simple contagion
nsamples = 50000
skip = 10

sc = lambda nu, b: 1 - (1 - b) ** nu
c = sc(np.arange(n), b)

x1 = contagion_process(A, gamma, c, x0, tmin=0, tmax=tmax)

In [ ]:
plt.spy(x1.T)

In [ ]:
p_c = np.ones((2, n))
p_rho = np.array([1, 1])

A0 = erdos_renyi(n, 0)
samples1, l = infer_adjacency_matrix(
    x1, A0, p_rho, p_c, nsamples=nsamples, burn_in=0, skip=skip, return_likelihood=True
)

In [ ]:
plt.figure()
plt.title("Simple contagion")
plt.subplot(211)
for i in range(0, int(len(samples1) / 2), 1000):
    h = []
    idx = []
    for j in range(i, len(samples1)):
        h.append(hamming_distance(samples1[i], samples1[j]))
        idx.append(j * skip)
    plt.plot(
        idx,
        h,
        label=f"Distance from iteration {i}",
        linewidth=0.5,
        color=plt.cm.RdBu((2 * i / len(samples1))),
    )
    print(f"plot {i} finished")
plt.xlabel("Iteration number")
plt.ylabel("Hamming distance")

plt.subplot(212)
burn_in = 100000
burn_in_idx = int(burn_in / skip)
h = []
idx = []
for j in range(burn_in_idx, len(samples1)):
    h.append(hamming_distance(samples1[burn_in_idx], samples1[j]))
    idx.append((j - burn_in_idx) * skip)
plt.plot(idx, h, color=plt.cm.RdBu((2 * burn_in_idx / len(samples1))))
plt.xlim([0, 50000])
plt.xlabel("decorrelation steps")
plt.xlabel("Hamming distance")
plt.tight_layout()
# plt.savefig("Figures/simple_contagion_distances.png", dpi=1000)
# plt.savefig("Figures/simple_contagion_distances.pdf", dpi=1000)

In [ ]:
plt.imshow(samples1.mean(axis=0))

In [ ]:
# threshold contagion
tau = 2
cc = lambda nu, tau, beta: beta * (nu >= tau)
b = 0.2
c = cc(np.arange(n), tau, b)

x2 = contagion_process(A, gamma, c, x0, tmin=0, tmax=tmax, random_seed=None)

In [ ]:
plt.spy(x2.T)

In [ ]:
p_c = np.ones((2, n))
p_rho = np.array([2, 5])
A0 = erdos_renyi(n, 0)

samples2, l = infer_adjacency_matrix(
    x2, A0, p_rho, p_c, nsamples=nsamples, burn_in=0, skip=skip, return_likelihood=True
)

In [ ]:
plt.figure()
plt.title("Threshold contagion")
plt.subplot(211)
for i in range(0, int(len(samples2) / 2), 1000):
    h = []
    idx = []
    for j in range(i, len(samples2)):
        h.append(hamming_distance(samples2[i], samples2[j]))
        idx.append(j * skip)
    plt.plot(
        idx,
        h,
        label=f"Distance from iteration {i}",
        linewidth=0.5,
        color=plt.cm.RdBu((2 * i / len(samples2))),
    )
    print(f"plot {i} finished")
plt.title("Threshold contagion")
plt.xlabel("Iteration number")
plt.ylabel("Hamming distance")

plt.subplot(212)
burn_in = 100000
burn_in_idx = int(burn_in / skip)
h = []
idx = []
for j in range(burn_in_idx, len(samples2)):
    h.append(hamming_distance(samples2[burn_in_idx], samples2[j]))
    idx.append((j - burn_in_idx) * skip)
plt.plot(idx, h, color=plt.cm.RdBu((2 * burn_in_idx / len(samples2))))
plt.xlim([0, 50000])
plt.xlabel("decorrelation steps")
plt.xlabel("Hamming distance")
plt.tight_layout()
# plt.savefig("Figures/threshold_contagion_distances.png", dpi=1000)
# plt.savefig("Figures/threshold_contagion_distances.pdf", dpi=1000)

In [ ]:
plt.imshow(samples2.mean(axis=0))